In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
        
clearall()

In [2]:
############################
######## LIBRAIRIES ########
############################
print('Importing Librairies...')
from sklearn.linear_model import LinearRegression
import colorama as col
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import time
import os
import datetime as dt
import numpy as np
import fxcmpy
import socketio
import shelve
import talib
import statistics
import pyttsx3

engine = pyttsx3.init()

print('Librairies imported\n')
print('version fxcmpy :',fxcmpy.__version__)
%matplotlib inline
plt.style.use('seaborn')
engine.say("librairie loaded")
engine.runAndWait()
print('Prêt')

Importing Librairies...
Librairies imported

version fxcmpy : 1.2.6
Prêt


In [3]:
engine.say("Ignition du Lumberjack")
engine.runAndWait() 

##############################################
######## INITIALISATION DES VARIABLES ########
##############################################


# COMPTE TEST #

TOKEN = '89410007226e1ad8ffce5dbf03f1e1bf056f2451'
server = 'demo'
user_id = 'D261219574'
compte = '01215057'
password = '3555'


'''
# COMPTE DEV #
TOKEN = '79f83cbff13d296eb6d9b6c1ed6dccd768ef925a'
server = 'demo'
user_id = 'D261219577'
compte = '01215060'
password = '3877'
'''

'''
# COMPTE POUBELLE #
TOKEN = 'a66bf5c0ec732487f4b348baf6364d3ee8784935'
server = 'demo'
user_id = 'D261219576'
compte = '01215059'
password = '8029'
'''


# minutes: 'm1' , 'm5' , 'm15' , 'm30' 
# hours: 'H1' , 'H2', 'H3', 'H4', 'H6' 'H8'
# one day: 'D1'
# one week: 'W1'
# one month: 'M1'


_number = 40
_period = 'm5'
_period2 = 'D1'

_amount = 10000
_amount = _amount / 1000

_target = 0.0020
_sl = 0.0090
_spread_filter = 0.0005

engine.say("בסדר")
engine.runAndWait()
print('Variables initialisées')

Variables initialisées


In [4]:

############################
######## FONCTIONS ########
###########################


def conX():
    con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error',server=server)
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
    return(con)

def deconX():
    con = con.close()
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion non intérrompue'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion intérrompue'+col.Style.RESET_ALL)
    return()


def scrap_hist(_ticker,number,laperiod):
    print(col.Fore.BLUE,'\rScraping des données OHLC pour le ticker',col.Fore.YELLOW,_ticker,col.Style.RESET_ALL,' --- ',\
          dt.datetime.now().strftime(format='%Y-%m-%d %H:%M:%S'),end='',flush=True)
    time.sleep(0.2)
    data = con.get_candles(instrument=_ticker,number = 40,period=laperiod)
    df = pd.DataFrame(index=data.index)
    df['Open_Bid'] = data['bidopen']
    df['Open_Ask'] = data['askopen']
    df['Open'] = (data['bidopen']+data['askopen'])/2
    df['High_Bid'] = data['bidhigh']
    df['High_Ask'] = data['askhigh']
    df['High'] = (data['bidhigh']+data['askhigh'])/2
    df['Low_Bid'] = data['bidlow']
    df['Low_Ask'] = data['asklow']
    df['Low'] = (data['bidlow']+data['asklow'])/2
    df['Close_Bid'] = data['bidclose']
    df['Close_Ask'] = data['askclose']
    df['Close'] = (data['bidclose']+data['askclose'])/2
    return(df)



def Short(ticker,_mise):
    print('')
    print(col.Fore.RED,'Short (',col.Fore.YELLOW,' Ticker : ',ticker,col.Fore.WHITE,' - Amount : ',_amount,')',col.Style.RESET_ALL)
    engine.say("Foxtrot Foxtrot. We are going short")
    engine.runAndWait()
    '''    
    p0os = con.create_entry_order(symbol=ticker,\
                is_buy=False,\
                    amount=_mise,\
                    time_in_force='GTD',\
                        order_type='Entry',\
                        rate=round(float(con.get_last_price(ticker).Bid * 0.9999),4),\
                            is_in_pips=False,\
                                limit=round(float(con.get_last_price(ticker).Bid * (1 - _target)),4) ,\
                                    stop=round(float(con.get_last_price(ticker).Bid * (1 + _sl)),4),\
                                        trailing_step=None,\
                                            expiration = (dt.datetime.now() + dt.timedelta(minutes=10)).strftime(format='%Y-%m-%d %H:%M'))
                                    
        
    '''    

    p0os= con.open_trade(symbol=ticker, is_buy=False, is_in_pips=False, amount=_mise, time_in_force='GTC', order_type='AtMarket',\
                                limit=round(float(con.get_last_price(ticker).Bid * (1 - _target)),4) ,\
                         stop=round(float(con.get_last_price(ticker).Bid * (1 + _sl)),4),\
                                        trailing_step=None)
    return(p0os)

def Long(ticker,_mise):
    print('')
    print(col.Fore.GREEN,'Long (',col.Fore.YELLOW,' Ticker : ',ticker,col.Fore.WHITE,' - Amount : ',_amount,')',col.Style.RESET_ALL)
    engine.say("Foxtrot Foxtrot. We are going long")
    engine.runAndWait()
    
    '''
    p0os = con.create_entry_order(symbol=ticker,\
                is_buy=True,\
                    amount=_mise,\
                    time_in_force='GTD',\
                        order_type='Entry',\
                        rate=round(float(con.get_last_price(ticker).Ask * 1.0001),4),\
                            is_in_pips=False,\
                                limit=round(float(con.get_last_price(ticker).Ask * (1 + _target)),4) ,\
                                    stop=round(float(con.get_last_price(ticker).Ask * (1 - _sl)),4),\
                                        trailing_step=None,\
                                            expiration = (dt.datetime.now() + dt.timedelta(minutes=10)).strftime(format='%Y-%m-%d %H:%M'))
    '''
    p0os= con.open_trade(symbol=ticker, is_buy=True, is_in_pips=False, amount=_mise, time_in_force='GTC', order_type='AtMarket',\
                                limit=round(float(con.get_last_price(ticker).Ask * (1 + _target)),4) ,\
                         stop=round(float(con.get_last_price(ticker).Ask * (1 - _sl)),4),\
                                        trailing_step=None)
    
    return(p0os)
 

engine.say("functions initiated")
engine.runAndWait()
print('Functions initiated')

Functions initiated


In [5]:
#########################
### CONNEXION A L'API ###
#########################

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor for FOR_EX\nLumberJack Jyss 5780(c)')
print(col.Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(col.Fore.YELLOW,col.Back.BLUE,'--- Go!em STOCHASTIC-RSI #v0.14 PAPER TRADE ---',col.Style.RESET_ALL)

print('')
engine.say(" Initialisation du Gaulem Stok R S E paper trade version 0.14")
engine.say("Connexion du Gaulem hà la Péh e")
engine.runAndWait()

try:
    con.is_connected() == True
    
    engine.say("already Connected")
    engine.runAndWait()
    print(col.Fore.GREEN+'Connexion rétablie'+col.Style.RESET_ALL)
    print('Compte utilisé : ',con.get_account_ids())
    print('')
    
except:
    try:
        con = conX()
        con.is_connected() == True
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
        engine.say("Connected")
        engine.runAndWait()
    except:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
        engine.say("Not Connected, sal rass de mor, pitun cé la merd")
        engine.say("vérifi ton internet, é relance le gaulèm")
        engine.runAndWait()
        print('')
        #os._exit(0)
        con = deconX()
        time.sleep(1)
        con = conX()

Global Optimized LumberJack Environment Motor for FOR_EX
LumberJack Jyss 5780(c)
 °0Oo_D.A.G._26_oO0°
  --- Go!em STOCHASTIC-RSI #v0.14 PAPER TRADE --- 



|ERROR|2020-09-01 11:09:44,843|Connection error: Not authorized


Connexion non établie



UnboundLocalError: local variable 'con' referenced before assignment

In [ ]:
%%time

#TICKERS = con.get_instruments()
TICKERS = [ 'EUR/USD','USD/JPY','GBP/USD','USD/CHF','EUR/CHF','AUD/USD','USD/CAD','NZD/USD','EUR/GBP','EUR/JPY','GBP/JPY','CHF/JPY','GBP/CHF','EUR/AUD','EUR/CAD','AUD/CAD','AUD/JPY','CAD/JPY','NZD/JPY','GBP/CAD','GBP/NZD','GBP/AUD','AUD/NZD','USD/SEK','EUR/SEK','EUR/NOK','USD/NOK','USD/MXN','AUD/CHF','EUR/NZD','USD/ZAR','ZAR/JPY','NZD/CHF','CAD/CHF','NZD/CAD','USD/ILS','USD/CNH']

engine.say("Ready to go? Prepare to be billionnaire!!!! Bim")
engine.runAndWait()
i = -1
for loop in range(72000000):
    for x in range(1000):
        while dt.datetime.now().minute not in [0,5,10,15,20,25,30,35,40,45,50,55]:
            time.sleep(1)
    for x in TICKERS:
        if con.is_subscribed(x) == False:
            con.subscribe_market_data(x)

        data = scrap_hist(x,_number,_period)
        data2 = scrap_hist(x,_number,_period2)
        price = data.copy()
        price2 = data2.copy()
        
        MM21 = talib.EMA(price.Close, timeperiod=21)
        rsi = talib.RSI(price.Close, timeperiod=14)
        rsi_high = 65
        rsi_low = 35
        upperband, middleband, lowerband = talib.BBANDS(price.Close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
        fastk, fastd = talib.STOCHRSI(price.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)#STOCHASTICRSI
        slowk, slowd = talib.STOCH(price.High, price.Low, price.Close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)#STOCHASTIC
        
        MM82 = talib.EMA(price2.Close, timeperiod=8)
        MM212 = talib.EMA(price2.Close, timeperiod=21)
        MM342 = talib.EMA(price2.Close, timeperiod=34)
        rsi2 = talib.RSI(price2.Close, timeperiod=14)
        upperband2, middleband2, lowerband2 = talib.BBANDS(price2.Close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
        fastk2, fastd2 = talib.STOCHRSI(price2.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)#STOCHASTICRSI
        slowk2, slowd2 = talib.STOCH(price2.High, price2.Low, price2.Close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

        if slowk[i-1] > 80 and slowk[i-1] > slowd[i-1] and slowk[i] < slowd[i] and price.High[i-1] > price.High[i-2] and price.High[i] < price.High[i-2] :
            j = len(price2[price2.index <= price.index[i]])-1
                
            if  price2.Close[j] < MM82[j] and MM82[j]<MM212[j] and MM212[j] < MM342[j] \
                and (price.Close_Ask[i]-price.Close_Bid[i])/price.Close_Ask[i] < _spread_filter:

                '''
                if con.get_open_positions().shape[0] == 0:
                    p0os = Short(x,_amount)
                
                elif x not in list(con.get_open_positions().currency):
                    p0os = Short(x,_amount)
                '''
                p0os = Short(x,_amount)
                           
        elif slowk[i-1] < 20 and slowk[i-1] < slowd[i-1] and slowk[i] > slowd[i] and price.Low[i-1] < price.Low[i-2] and price.Low[i] > price.Low[i-2]:
            j = len(price2[price2.index <= price.index[i]])-1
            
            if price2.Close[j] > MM82[j] and MM82[j]>MM212[j] and MM212[j] > MM342[j] \
                and (price.Close_Ask[i]-price.Close_Bid[i])/price.Close_Ask[i] < _spread_filter:

                '''
                if con.get_open_positions().shape[0] == 0:
                    p0os = Long(x,_amount)
                
                elif x not in list(con.get_open_positions().currency):
                    p0os = Long(x,_amount)
                '''
                p0os = Long(x,_amount)

        else :
            pass
    print('\nPause après le loop',loop,'  ',dt.datetime.now().strftime(format='%Y-%m-%d- %H:%M:%S'),'\n')
    engine.say("\nNow, entering the pause\n")
    engine.runAndWait()
    time.sleep(60)
            
